# Refer to 
https://github.com/Azure/ViennaDocs/blob/master/PrivatePreview/notebooks/01.train-within-notebook.ipynb
https://github.com/Azure/ViennaDocs/blob/master/PrivatePreview/notebooks/11.production-deploy-to-aks.ipynb

In [1]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model

In [2]:
import azureml.core
print(azureml.core.VERSION)

0.1.65


In [3]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

Found the config file in: /home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow/aml_config/config.json
jayavienna
jayavienna
eastus2
edf507a2-6235-46c5-b560-fd463ba2e771


In [4]:
#Creating the model pickle file
import tensorflow as tf
from resnet152 import ResNet152
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input, decode_predictions

model = ResNet152(weights='imagenet')

Using TensorFlow backend.


In [5]:
model

In [6]:
model_json = model.to_json()
with open("model_resnet.json", "w") as json_file:
    json_file.write(model_json)

In [7]:
model.save_weights("model_resnet_weights.h5")

In [8]:
#from keras.models import model_from_json
#model = model_from_json(model_json)

In [9]:
model.load_weights('model_resnet_weights.h5')

In [10]:
model

In [11]:
#Register the model
#from azureml.core.model import Model
#model = Model.register(model_path = "model_resnet_weights.h5", # this points to a local file
#                       model_name = "resnet_model", # this is the name the model is registered as
#                       tags = ["dl", "resnet"],
#                       description = "resnet 152 model",
#                       workspace = ws)

#print(model.name, model.description, model.version)

In [12]:
#define init() function
def init():
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging

    global model
    model = ResNet152(weights='imagenet')
    print('Model loaded')

In [13]:
init()

Model loaded


In [14]:
#define run() function 
def run(img_path):
    
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging   
    
    model = ResNet152(weights='imagenet')
    print('Model loaded')
  
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    preds = model.predict(img)
    print('Predicted:', decode_predictions(preds, top=3))

In [15]:
import wget
wget.download('https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Lynx_lynx_poing.jpg/220px-Lynx_lynx_poing.jpg')

'220px-Lynx_lynx_poing (3).jpg'

In [16]:
img_path = '220px-Lynx_lynx_poing.jpg'

In [17]:
run(img_path)

Model loaded
Predicted: [[('n02127052', 'lynx', 0.99590886), ('n02128385', 'leopard', 0.001150445), ('n02123159', 'tiger_cat', 0.0009417962)]]


In [18]:
%%writefile score.py
def init():
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging

    global model
    model = ResNet152(weights='imagenet')
    print('Model loaded')
    
def run(img_path):
    
    import tensorflow as tf
    from resnet152 import ResNet152
    from keras.preprocessing import image
    from keras.applications.imagenet_utils import preprocess_input, decode_predictions

    import numpy as np
    import timeit as t
    import base64
    import json
    from PIL import Image, ImageOps
    from io import BytesIO
    import logging   
    
    model = ResNet152(weights='imagenet')
    print('Model loaded')
  
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    
    preds = model.predict(img)
    print('Predicted:', decode_predictions(preds, top=3))    

Overwriting score.py


In [19]:
%run score.py

In [ ]:
!more score.py

In [ ]:
!more conda_dependencies.yml

In [20]:
import os
os.getcwd()

'/home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow'

In [21]:
os.listdir(os.getcwd())

['AML_00_configuration_v0.1.50.ipynb',
 '00_DevelopModel.ipynb',
 'AML_02_DevelopModelDriver_AKS-old.ipynb',
 '220px-Lynx_lynx_poing (3).jpg',
 'AML_01_trainlocal-docker.ipynb',
 'model_resnet_weights.h5',
 'AML_02_resnet_model_weights.ipynb',
 '01_DevelopModelDriver.ipynb',
 '__pycache__',
 'samples',
 'resnet152_n.py',
 '06_SpeedTestWebApp.ipynb',
 '03_TestLocally.ipynb',
 'AML_02_Develop_init_run_functions.ipynb',
 '220px-Lynx_lynx_poing (1).jpg',
 'AML_02_AKS_deployment_v0.1.65.ipynb',
 'AML_02_AKS_deployment.ipynb',
 'aml_config',
 '05_TestWebApp.ipynb',
 '04_DeployOnAKS.ipynb',
 'score.py',
 'model_resnet.json',
 'AML_01_trainlocal_v0.1.50.ipynb',
 'AML_00_configuration_v0.1.65.ipynb',
 '02_BuildImage.ipynb',
 '07_TearDown.ipynb',
 'helpers.py',
 'testing_utilities.py',
 '220px-Lynx_lynx_poing.jpg',
 'config',
 'conda_dependencies.yml',
 'resnet152.py',
 'AML_02_AKS_deployment_v0.1.50.ipynb',
 '220px-Lynx_lynx_poing (2).jpg',
 'README.md',
 'environment.yml',
 'sample_projects',


In [1]:
from azureml.core.image import ContainerImage

In [2]:
ContainerImage.create??

In [22]:


image_config = ContainerImage.image_configuration(execution_script = "score.py",
                                                  runtime = "python",
                                                  conda_file = "conda_dependencies.yml",
                                                  description = "Image for AKS Deployment Tutorial",
                                                  tags = {"name":"AKS","project":"AML"}, 
                                                  dependencies = ["resnet152.py","220px-Lynx_lynx_poing.jpg"],
                                                  enable_gpu = True
                                                 )

image = ContainerImage.create(name = "myimage1",
                              # this is the model object
                              models = [],                              
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

Creating image
Running........................................................................................
SucceededImage creation operation finished for image myimage1:11, operation "Succeeded"


In [23]:
#list images
images = ws.images()
images

{'my-aci-svc-1': ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x7fb578f80978>, name=my-aci-svc-1, id=my-aci-svc-1:4, tags={}, properties={}, version=4),
 'myimage1': ContainerImage(workspace=<azureml.core.workspace.Workspace object at 0x7fb578f80978>, name=myimage1, id=myimage1:11, tags={'name': 'AKS', 'project': 'AML'}, properties={}, version=11)}

In [24]:
#for img in ws.images():
#    if img.name == 'myimage1': img.delete()

In [25]:
#Provision AKS cluster
# Use the default configuration (can also provide parameters to customize)
#prov_config = AksCompute.provisioning_configuration()

#aks_name = 'jaya-aks-7' 
# Create the cluster
#aks_target = ComputeTarget.create(workspace = ws, 
#                                  name = aks_name, 
#                                  provisioning_configuration = prov_config)

In [26]:
#Provision AKS cluster with GPU machine
# Use the default configuration (can also provide parameters to customize)
prov_config = AksCompute.provisioning_configuration(vm_size='Standard_NC6')

aks_name = 'jaya-aks-9' 
# Create the cluster
aks_target = ComputeTarget.create(workspace = ws, 
                                  name = aks_name, 
                                  provisioning_configuration = prov_config)

In [27]:
%%time
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating......................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None
CPU times: user 1.99 s, sys: 122 ms, total: 2.11 s
Wall time: 11min 39s


In [28]:
!az aks get-credentials -n 'jaya-aks-8' -g jayavienna -a -f config

The Resource 'Microsoft.ContainerService/managedClusters/jaya-aks-8' under resource group 'jayavienna' was not found.


In [29]:
!kubectl --kubeconfig config get services

Error from server (NotAcceptable): unknown (get services)


In [30]:
#Deploy web service to AKS
#Set the web service configuration (using default here)
aks_config = AksWebservice.deploy_configuration()

In [31]:
%%time
aks_service_name ='jaya-aks-service-10'

aks_service = Webservice.deploy_from_image(workspace = ws, 
                                           name = aks_service_name,
                                           image = image,
                                           deployment_config = aks_config,
                                           deployment_target = aks_target)
aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

Creating service
Running.......................................................................................................................
TimedOutAKS service creation operation finished, operation "TimedOut"
Transitioning
CPU times: user 1.94 s, sys: 139 ms, total: 2.08 s
Wall time: 10min 28s


In [32]:
#Debug

# load workspace from default config file
ws = Workspace.from_config()

# list all web services in the workspace
#for s in ws.webservices():
#    print(s.name)

# instantiate the web service object from workspace and service name
svc = Webservice(workspace = ws, name = 'jaya-aks-service-10')

# print out the Docker log
print(svc.get_logs())

Found the config file in: /home/jayaubuntudsvm/Desktop/AKSDeploymentTutorial/Keras_Tensorflow/aml_config/config.json


WebserviceException: Received bad response from Model Management Service:
Response Code: 500
Headers: {'Date': 'Thu, 18 Oct 2018 19:19:03 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'api-supported-versions': '2018-03-01-preview', 'x-ms-client-request-id': '9b132e62ebdb45b3b92631e9b9fba8b0', 'x-ms-client-session-id': '', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"InternalServerError","statusCode":500,"message":"An internal server error occurred. Please try again. If the problem persists, contact support"}'

In [33]:
#debug web service failure
print(ws.webservices()['jaya-aks-service-10'].get_logs())

WebserviceException: Received bad response from Model Management Service:
Response Code: 500
Headers: {'Date': 'Thu, 18 Oct 2018 19:19:20 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'api-supported-versions': '2018-03-01-preview', 'x-ms-client-request-id': '7c211a1922554d6da4a0f6ccfc1778f3', 'x-ms-client-session-id': '', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains; preload'}
Content: b'{"code":"InternalServerError","statusCode":500,"message":"An internal server error occurred. Please try again. If the problem persists, contact support"}'

In [ ]:
#clean up resources
#aks_target = AksCompute(name='jaya-aks-1',workspace=ws)
#aks_target.delete()

In [ ]:
#alternate code to clean up resources
#!az aks delete --resource-group jayavienna --name jaya-aks-2 --yes

In [ ]:
#for s in ws.webservices():
#    print(s.name)

In [ ]:
#s =  Webservice(ws, 'jaya-aks-service-2')
#s.delete()

In [ ]:
#from azureml.core import Workspace
#from azureml.core.compute import AksCompute, ComputeTarget

#ws = Workspace.from_config()

#for c in ws.compute_targets():
#    print(c.name)
